In [6]:
!pip3 install catboost

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 6.6 MB/s eta 0:00:000m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 2.6 MB/s eta 0:00:00


In [2]:
!unzip titanic.zip

Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [7]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [8]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [9]:
test_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [19]:
train_df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [14]:
# get average age in train/test
avg_age = (train_df["Age"].mean() + test_df["Age"].mean()) / 2
avg_age

29.985854004252303

In [15]:
# Fill age with mean
train_df[['Age']] = train_df[['Age']].fillna(value=avg_age)
test_df[['Age']] = test_df[['Age']].fillna(value=avg_age)

In [18]:
# Fill cabin to -999
train_df.fillna(-999,inplace=True)
test_df.fillna(-999,inplace=True)

In [20]:
x = train_df.drop('Survived',axis=1) # train df without survived status
y = train_df.Survived # df with passenger id and survived status

In [21]:
x.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [22]:
# Convert floats to ints, because CatBoost can use floats
train_df = train_df.astype({"Age": int, "Fare": int})
test_df = test_df.astype({"Age": int, "Fare": int})

In [23]:
import numpy as np

cate_features_index = np.where(x.dtypes != float)[0]

In [24]:
cate_features_index

array([ 0,  1,  2,  3,  5,  6,  7,  9, 10])

In [25]:
from sklearn.model_selection import train_test_split
# Valid data
xtrain,xtest,ytrain,ytest = train_test_split(x,y,train_size=.85,random_state=1234)


In [79]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=42, depth=3, loss_function='Logloss')

In [80]:
model.fit(xtrain,ytrain,cat_features=cate_features_index,eval_set=(xtest,ytest))

Learning rate set to 0.029583
0:	learn: 0.7873184	test: 0.7910448	best: 0.7910448 (0)	total: 2.69ms	remaining: 2.69s
1:	learn: 0.7886394	test: 0.7910448	best: 0.7910448 (0)	total: 5.69ms	remaining: 2.84s
2:	learn: 0.7859974	test: 0.7910448	best: 0.7910448 (0)	total: 8.07ms	remaining: 2.68s
3:	learn: 0.7859974	test: 0.7910448	best: 0.7910448 (0)	total: 9.95ms	remaining: 2.48s
4:	learn: 0.7992074	test: 0.8134328	best: 0.8134328 (4)	total: 11.8ms	remaining: 2.35s
5:	learn: 0.7886394	test: 0.8134328	best: 0.8134328 (4)	total: 14ms	remaining: 2.32s
6:	learn: 0.7965654	test: 0.8134328	best: 0.8134328 (4)	total: 15.9ms	remaining: 2.26s
7:	learn: 0.7978864	test: 0.8134328	best: 0.8134328 (4)	total: 18.2ms	remaining: 2.26s
8:	learn: 0.8005284	test: 0.8134328	best: 0.8134328 (4)	total: 20ms	remaining: 2.21s
9:	learn: 0.7992074	test: 0.8134328	best: 0.8134328 (4)	total: 21.8ms	remaining: 2.15s
10:	learn: 0.8005284	test: 0.8134328	best: 0.8134328 (4)	total: 23.9ms	remaining: 2.15s
11:	learn: 0.800

In [81]:
from sklearn.metrics import accuracy_score

print('the test accuracy is :{:.6f}'.format(accuracy_score(ytest,model.predict(xtest))))

the test accuracy is :0.850746


In [82]:
params = model.get_params()
#params["loss_funcion"] = "Logloss"
params

{'depth': 3,
 'loss_function': 'Logloss',
 'random_seed': 42,
 'use_best_model': True,
 'eval_metric': 'Accuracy'}

In [ ]:
# Cross val
from catboost import Pool, cv

cv_data = cv(Pool(x,y,cat_features=cate_features_index), params,fold_count=10)


Training on fold [0/10]
0:	learn: 0.7927591	test: 0.7111111	best: 0.7111111 (0)	total: 13.5ms	remaining: 13.5s
1:	learn: 0.7952559	test: 0.7111111	best: 0.7111111 (0)	total: 31.1ms	remaining: 15.5s
2:	learn: 0.7902622	test: 0.7555556	best: 0.7555556 (2)	total: 36.7ms	remaining: 12.2s
3:	learn: 0.8052434	test: 0.7888889	best: 0.7888889 (3)	total: 49.9ms	remaining: 12.4s
4:	learn: 0.8127341	test: 0.7888889	best: 0.7888889 (3)	total: 57.3ms	remaining: 11.4s
5:	learn: 0.8127341	test: 0.7888889	best: 0.7888889 (3)	total: 66.1ms	remaining: 10.9s
6:	learn: 0.8102372	test: 0.7888889	best: 0.7888889 (3)	total: 75.3ms	remaining: 10.7s
7:	learn: 0.8102372	test: 0.7888889	best: 0.7888889 (3)	total: 84.6ms	remaining: 10.5s
8:	learn: 0.8127341	test: 0.7888889	best: 0.7888889 (3)	total: 94.1ms	remaining: 10.4s
9:	learn: 0.8127341	test: 0.7888889	best: 0.7888889 (3)	total: 97.4ms	remaining: 9.64s
10:	learn: 0.8127341	test: 0.7888889	best: 0.7888889 (3)	total: 101ms	remaining: 9.12s
11:	learn: 0.812734

In [ ]:
cv_data

In [55]:
pred = model.predict(test_df)
pred = pred.astype(np.int)
submission = pd.DataFrame({'PassengerId':test_df['PassengerId'],'Survived':pred})
submission.to_csv('catboost_solution.csv',index=False)


/tmp/ipykernel_1055027/296897289.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pred = pred.astype(np.int)
